In [ ]:
import requests
import json
from datetime import datetime, timedelta

url = "https://kystdatahuset.no/ws/api/ais/positions/within-geom-time"
headers = {
    "Accept": "text/plain",
    "Content-Type": "application/json"
}

start_date = datetime.strptime("2021-07-01T00:00:00", "%Y-%m-%dT%H:%M:%S")
end_date = datetime.strptime("2022-03-01T00:00:00", "%Y-%m-%dT%H:%M:%S")

# Define the interval (1 day in this case)
interval = 1

intervals = []
current_start_date = start_date
while current_start_date < end_date:
    current_end_date = min(current_start_date + timedelta(days=interval), end_date)
    intervals.append((current_start_date, current_end_date))
    current_start_date = current_end_date

# Make requests for each interval and save responses to separate json files
for start, end in intervals:
    payload = {
        "geom": "POLYGON((17.158265 76.77251,17.158265 79.45649,7.910835 79.45649,7.910835 76.77251,17.158265 76.77251))",
        "start": start.strftime("%Y-%m-%dT%H:%M:%S"),
        "end": end.strftime("%Y-%m-%dT%H:%M:%S"),
        "minSpeed": 0.0
    }
    
    json_data = json.dumps(payload)
    response = requests.post(url, headers=headers, data=json_data)
    
    if response.status_code == 200:
        file_name = f'ais_data_{start.strftime("%Y%m%d")}_to_{end.strftime("%Y%m%d")}.json'
        
        with open(file_name, 'w') as f:
            json.dump(response.json(), f)
        
        print(f"Data for period {start} to {end} successfully written to {file_name}")
    else:
        print(f"Failed to retrieve data for period {start} to {end}. Status Code: {response.status_code}")


In [ ]:
#ROI in Svalbard
import folium
from shapely.geometry import Polygon

coordinates = [(17.158265, 76.77251), (17.158265, 79.45649), (7.910835, 79.45649), (7.910835, 76.77251), (16.558265, 76.77251)]

polygon = Polygon(coordinates)

m = folium.Map(location=[sum(y for _, y in coordinates) / len(coordinates), 
                          sum(x for x, _ in coordinates) / len(coordinates)], zoom_start=8)

folium.Polygon(locations=[list(coord)[::-1] for coord in polygon.exterior.coords],
               color='blue', 
               fill=True,
               fill_color='blue').add_to(m)

# Display the map
m.save('polygon_map.html')
m


In [ ]:
#Combine previous json files into a common excel file

import glob
import pandas as pd
import json

directory = "."

json_files = glob.glob(f"{directory}/ais_data_*.json")

combined_df = pd.DataFrame()

column_names = [
    "mmsi",
    "date time utc",
    "longitude",
    "latitude",
    "course over ground",
    "speed over ground",
    "AIS message number",
    "calc_speed",
    "seconds to previous point",
    "distance to previous point"
]

# Loop through each JSON file
for file_path in json_files:
    with open(file_path, 'r') as file:
        data = json.load(file)
    data_section = data['data']
    df = pd.DataFrame(data_section, columns=column_names)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

output_excel_file = "ais_data_split.xlsx"

# Excel row limit
row_limit = 1048576

with pd.ExcelWriter(output_excel_file, engine='xlsxwriter') as writer:
    start_row = 0
    for start in range(0, combined_df.shape[0], row_limit):
        end = min(start + row_limit, combined_df.shape[0])
        combined_df.iloc[start:end].to_excel(writer, index=False, sheet_name=f'Sheet{start//row_limit + 1}')
        
print(f"Data from all files has been successfully combined and saved to {output_excel_file}.")

In [ ]:
#preprocessing

def split_on_duration(group, min_duration=pd.Timedelta(minutes=10), max_duration=pd.Timedelta(hours=12)):
    group['gap'] = group['date time utc'].diff() > pd.Timedelta(minutes=45)
    group['split_id'] = group['gap'].cumsum()
    segments = []
    
    for _, g in group.groupby('split_id'):
        # Check duration and split further if necessary
        duration = g['date time utc'].max() - g['date time utc'].min()
        if duration < min_duration:
            continue  # Skip trajectories that are too short
        while duration > max_duration:
            # Split trajectory at the maximum duration limit
            max_time = g['date time utc'].min() + max_duration
            segment = g[g['date time utc'] <= max_time]
            segments.append(segment)
            g = g[g['date time utc'] > max_time]
            duration = g['date time utc'].max() - g['date time utc'].min()
        segments.append(g)  # Append the remaining segment if any
    
    return segments

def process_trajectory(trajectory):
    if len(trajectory) > 360:
        trajectory = trajectory.iloc[::len(trajectory)//360]
    trajectory = trajectory.set_index('date time utc').resample('2T').mean().interpolate()
    return trajectory.reset_index()

def filter_trajectory_by_date(trajectory, start_date, end_date):
    """Filter the trajectory DataFrame to only include data between start_date and end_date."""
    return trajectory[(trajectory['date time utc'] >= start_date) & (trajectory['date time utc'] <= end_date)]

def process_all_sheets(excel_path, start_date, end_date):
    xls = pd.ExcelFile(excel_path)
    trajectories_by_mmsi = {}

    for sheet_name in xls.sheet_names:
        ais_data = pd.read_excel(xls, sheet_name=sheet_name)
        ais_data['date time utc'] = pd.to_datetime(ais_data['date time utc'])
        ais_data.sort_values(['mmsi', 'date time utc'], inplace=True)

        for mmsi, group in ais_data.groupby('mmsi'):
            split_trajectories = split_on_duration(group)
            processed_trajectories = [process_trajectory(traj) for traj in split_trajectories]
            # Filter trajectories by date
            filtered_trajectories = [filter_trajectory_by_date(traj, start_date, end_date) for traj in processed_trajectories]
            filtered_trajectories = [traj for traj in filtered_trajectories if not traj.empty]

            if mmsi in trajectories_by_mmsi:
                trajectories_by_mmsi[mmsi].extend(filtered_trajectories)
            else:
                trajectories_by_mmsi[mmsi] = filtered_trajectories

    return trajectories_by_mmsi

# Define the date range
start_date = pd.Timestamp('2021-07-01')
end_date = pd.Timestamp('2022-03-01')

excel_path = 'ais_data_split.xlsx'  
trajectories_by_mmsi = process_all_sheets(excel_path, start_date, end_date)

# Example: Print a specific trajectory for an MMSI of interest
mmsi_of_interest = 257524500  
if mmsi_of_interest in trajectories_by_mmsi and trajectories_by_mmsi[mmsi_of_interest]:
    print(trajectories_by_mmsi[mmsi_of_interest][0])  
else:
    print("No trajectories found for this MMSI.")


In [ ]:
writer = pd.ExcelWriter('trajectories_by_mmsi.xlsx', engine='xlsxwriter')

# Iterate through each MMSI's trajectories to save them in separate sheets
for mmsi, trajectories in trajectories_by_mmsi.items():
    for i, trajectory in enumerate(trajectories):
        sheet_name = f'MMSI_{mmsi}_Traj_{i+1}'
        # Ensure the sheet_name is not longer than 31 characters
        sheet_name = sheet_name[:31]
        trajectory.to_excel(writer, sheet_name=sheet_name, index=False)


writer.close()

In [ ]:
#Statistics

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees).
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of Earth in kilometers
    return c * r

def calculate_distance(trajectory):
    distances = [haversine(lat1, lon1, lat2, lon2) for (lat1, lon1), (lat2, lon2) 
                 in zip(trajectory[['latitude', 'longitude']].values[:-1], 
                         trajectory[['latitude', 'longitude']].values[1:])]
    return sum(distances)

def calculate_duration(trajectory):
    start_time = trajectory['date time utc'].iloc[0]
    end_time = trajectory['date time utc'].iloc[-1]
    return (end_time - start_time) / pd.Timedelta(hours=1)

trajectory_durations = []
distances_covered = []
average_speeds = []

for mmsi, trajectories in trajectories_by_mmsi.items():
    for trajectory in trajectories:
        duration = calculate_duration(trajectory)
        distance = calculate_distance(trajectory)
        # Store duration and distance
        trajectory_durations.append(duration)
        distances_covered.append(distance)
        # Calculate and store average speed if duration is not zero
        if duration > 0:
            average_speeds.append(distance / duration)
        else:
            average_speeds.append(0)

mean_duration = np.mean(trajectory_durations)
mean_distance_covered = np.mean(distances_covered)
mean_average_speed = np.mean(average_speeds)

print(f"Mean duration of trajectories: {mean_duration:.2f} hours")
print(f"Mean distance covered per trajectory: {mean_distance_covered:.2f} km")
print(f"Mean average speed per trajectory: {mean_average_speed:.2f} km/h")


In [ ]:
#Distribution of Trajectories' Durations

import matplotlib.pyplot as plt
import seaborn as sns  

sns.set(style="whitegrid")

plt.figure(figsize=(12, 8))

plt.hist(trajectory_durations, bins=30, alpha=0.75, color='royalblue', edgecolor='black')

plt.title('Distribution of Trajectory Durations', fontsize=18)
plt.xlabel('Duration (Hours)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
#format trajectories into a pickle file
file_path = 'trajectories_by_mmsi.xlsx'
xls = pd.ExcelFile(file_path)

all_trajectories = {}

for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name)
    
    trajectory_info = {
        'mmsi': df['mmsi'].iloc[0],
        'track_length': len(df),
        'lat': df['latitude'].tolist(),
        'lon': df['longitude'].tolist(),
        'speed': df['speed over ground'].tolist(),
        'course': df['course over ground'].tolist(),
        'timestamp': pd.to_datetime(df['date time utc']).astype(int) / 10**9  # Convert to UNIX timestamp
    }
    
    # Use sheet name or MMSI as the dictionary key
    all_trajectories[sheet_name] = trajectory_info

pd.to_pickle(all_trajectories, 'formatted_trajectories.pkl')


In [ ]:
# Print data from pickle file
all_trajectories = pd.read_pickle('formatted_trajectories.pkl')

def print_dataset(data):
    # Iterate through the dictionary and print key details
    for key, value in data.items():
        print(f"Trajectory: {key}")
        print(f"MMSI: {value['mmsi']}")
        print(f"Track Length: {value['track_length']}")
        print(f"Latitudes: {value['lat'][:5]}")  # Print first 5 latitudes for brevity
        print(f"Longitudes: {value['lon'][:5]}")  # Print first 5 longitudes
        print(f"Speeds: {value['speed'][:5]}")  
        print(f"Courses: {value['course'][:5]}")  
        print(f"Timestamps: {value['timestamp'][:5]}")  
        print("\n")  
print_dataset(all_trajectories)


In [ ]:
####Plotting some trajectories (can adjust time and mmsis)

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Polygon

def plot_ship_trajectories_within_polygon(excel_path, mmsi_list, polygon_wkt, start_date, end_date):
    area_polygon = gpd.GeoSeries.from_wkt([polygon_wkt])
    area_polygon.crs = "EPSG:4326"
    
    area_polygon = area_polygon.to_crs(epsg=3857)
    
    minx, miny, maxx, maxy = area_polygon.total_bounds
    
    xl = pd.ExcelFile(excel_path)
    
    fig, ax = plt.subplots(figsize=(15, 15))

    for mmsi in mmsi_list:
        sheets_for_mmsi = [sheet_name for sheet_name in xl.sheet_names if str(mmsi) in sheet_name]

        for sheet_name in sheets_for_mmsi:
            trajectory_df = xl.parse(sheet_name)
            trajectory_df['date time utc'] = pd.to_datetime(trajectory_df['date time utc'])
            trajectory_df = trajectory_df[(trajectory_df['date time utc'] >= start_date) & (trajectory_df['date time utc'] <= end_date)]

            if not trajectory_df.empty:
                gdf = gpd.GeoDataFrame(
                    trajectory_df, 
                    geometry=gpd.points_from_xy(trajectory_df['longitude'], trajectory_df['latitude'])
                )
                
                gdf.crs = "EPSG:4326"
                
                gdf = gdf.to_crs(epsg=3857)
                
            
                gdf.plot(ax=ax, marker='o', markersize=5, label=sheet_name, linewidth=0.1)

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_axis_off()

    plt.show()

# Usage example with date filtering
excel_path = 'trajectories_by_mmsi.xlsx' 
mmsi_list =['257301000', '257471500', '257489000', '273424080', '273425330', '273428090', '273428880', '273430750', '273431310', '273841710', '273845800', '518935000'] # Example MMSI numbers
polygon_wkt = "POLYGON((17.158265 76.77251,17.158265 79.45649,7.910835 79.45649,7.910835 76.77251,17.158265 76.77251))"
start_date = '2022-01-01'
end_date = '2022-02-08'
plot_ship_trajectories_within_polygon(excel_path, mmsi_list, polygon_wkt, start_date, end_date)


In [ ]:
####Plotting only the abnormal trajectory on 7 Jan 2022

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Polygon

def plot_ship_trajectories_within_polygon(excel_path, mmsi_list, polygon_wkt, start_date, end_date):
    area_polygon = gpd.GeoSeries.from_wkt([polygon_wkt])
    area_polygon.crs = "EPSG:4326"
    
    area_polygon = area_polygon.to_crs(epsg=3857)
    
    minx, miny, maxx, maxy = area_polygon.total_bounds
    
    xl = pd.ExcelFile(excel_path)
    
    fig, ax = plt.subplots(figsize=(15, 15))

    for mmsi in mmsi_list:
        sheets_for_mmsi = [sheet_name for sheet_name in xl.sheet_names if str(mmsi) in sheet_name]

        for sheet_name in sheets_for_mmsi:
            trajectory_df = xl.parse(sheet_name)
            trajectory_df['date time utc'] = pd.to_datetime(trajectory_df['date time utc'])
            trajectory_df = trajectory_df[(trajectory_df['date time utc'] >= start_date) & (trajectory_df['date time utc'] <= end_date)]

            if not trajectory_df.empty:
                gdf = gpd.GeoDataFrame(
                    trajectory_df, 
                    geometry=gpd.points_from_xy(trajectory_df['longitude'], trajectory_df['latitude'])
                )
                
                gdf.crs = "EPSG:4326"
                
                gdf = gdf.to_crs(epsg=3857)
                
            
                gdf.plot(ax=ax, marker='o', markersize=5, label=sheet_name, linewidth=0.1)

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_axis_off()

    plt.show()

# Usage example with date filtering
excel_path = 'trajectories_by_mmsi.xlsx' 
mmsi_list = ['273418680']
polygon_wkt = "POLYGON((17.158265 76.77251,17.158265 79.45649,7.910835 79.45649,7.910835 76.77251,17.158265 76.77251))"
start_date = '2022-01-01'
end_date = '2022-02-08'
plot_ship_trajectories_within_polygon(excel_path, mmsi_list, polygon_wkt, start_date, end_date)

In [ ]:
#checking all the mmsi 
total_trajectories = sum(len(trajectories) for trajectories in trajectories_by_mmsi.values())
trajectories_per_mmsi = {mmsi: len(trajectories) for mmsi, trajectories in trajectories_by_mmsi.items()}

total_trajectories, trajectories_per_mmsi

In [ ]:
#Storing ship types for every MMSI
full_ship_data = [
    (211336220, "Tanker"), (215753000, "Yacht"), (219001522, "Fishing Vessel"),
    (228092600, "Research Vessel"), (246337000, "Passenger Ship"), (257013250, "Other Type"),
    (257019000, "Patrol Vessel"), (257039490, "Cargo"), (257051200, "Patrol Vessel"),
    (257052200, "Patrol Vessel"), (257067270, "Patrol Vessel"), (257073260, "Other Type"),
    (257088040, "Other Type"), (257096620, "Passenger Ship"), (257153500, "Fishing Vessel"),
    (257230500, "Research Vessel"), (257275000, "Research Vessel"), (257301000, "Cargo"),
    (257471500, "Research Vessel"), (257524500, "Fishing Vessel"), (257564000, "Supply Ship"),
    (257582600, "Fishing Vessel"), (257620800, "Pleasure Craft"), (257677000, "Tanker"),
    (257729800, "HSC"), (257785000, "Cargo"), (258072000, "Supply Ship"),
    (258150000, "Lighthouse Vessel"), (258156500, "Other Type"), (258330730, "Other Type"),
    (258430000, "Tanker"), (259040000, "Patrol Vessel"), (259050000, "Patrol Vessel"),
    (259109000, "Fishing Vessel"), (259255000, "Cargo"), (259317000, "Patrol Vessel"),
    (261000150, "Research Vessel"), (273219900, "Cargo"), (273251100, "Cargo"),
    (273312270, "Fishing Vessel"), (273326830, "Fishing Vessel"), (273332880, "Fishing Vessel"),
    (273337950, "Cargo"), (273350610, "Fishing Vessel"), (273398180, "Fishing Vessel"),
    (273448120, "Fishing Vessel"), (273451570, "Fishing Vessel"), (273514800, "Fishing Vessel"),
    (273523100, "Fishing Vessel"), (273842800, "Fishing Vessel"), (352842000, "Passenger Ship"),
    (518100276, "Yacht"), (518935000, "Other Type"), (941004593, "Other Type"),
    (941004628, "Other Type"), (941213368, "Other Type"), (982575640, "Other Type"),
    (982575641, "Other Type"), (247405400, "Research Vessel"), (248290000, "Resolution 18 ship"),
    (250005461, "Cargo"), (257042110, "Other Type"), (257281000, "Fishing Vessel"),
    (257612000, "Cargo"), (257798800, "Cargo"), (257939500, "Fishing Vessel"),
    (257958900, "Pilot"), (258168000, "Other Type"), (261099070, "Port tender"),
    (265182000, "Ice breaker"), (273212100, "Fishing Vessel"), (273384970, "Cargo"),
    (273411400, "Research Vessel"), (273421520, "Fishing Vessel"), (273432030, "Cargo"),
    (273433220, "Fishing Vessel"), (273447010, "Fishing Vessel"), (273450850, "Fishing Vessel"),
    (273546000, "Other Type"), (276806000, "Fishing Vessel"), (276850000, "Sailing Vessel"),
    (952570116, "Other Type"), (246837000, "Cargo"), (257029990, "Other Type"),
    (257088070, "Passenger Ship"), (257113000, "Cargo"), (257139000, "Fishing Vessel"),
    (257193000, "Fishing Vessel"), (257489000, "Fishing Vessel"), (257702000, "Tanker"),
    (259247000, "Fishing Vessel"), (261208000, "Research Vessel"), (273213500, "Fishing Vessel"),
    (273218900, "Fishing Vessel"), (273313950, "Cargo"), (273317270, "Fishing Vessel"),
    (273341920, "Fishing Vessel"), (273348570, "Fishing Vessel"), (273352280, "Fishing Vessel"),
    (273392860, "Fishing Vessel"), (273415090, "Fishing Vessel"), (273428880, "Fishing Vessel"),
    (273433630, "Fishing Vessel"), (273841710, "Fishing Vessel"), (273890200, "Fishing Vessel"),
    (578001700, "Passenger Ship"), (257021000, "Other Type"), (257088020, "Military Ops"),
    (257934000, "Supply Ship"), (258107000, "Offshore Support Vessel"), (258535000, "Fishing Vessel"),
    (258828000, "Pollution Control Vessel"), (259987000, "Passenger Ship"), (273148810, "Cargo"),
    (273391950, "Fishing Vessel"), (273418680, "Fishing Vessel"), (273430750, "Fishing Vessel"),
    (276830000, "Fishing Vessel"), (224717000, "Fishing Vessel"), (257093370, "Fishing Vessel"),
    (257095980, "Passenger Ship"), (257105000, "Fishing Vessel"), (258499000, "Passenger Ship"),
    (259045000, "Military Ops"), (273213010, "Fishing Vessel"), (273213300, "Fishing Vessel"),
    (273217010, "Fishing Vessel"), (273311280, "Fishing Vessel"), (273317810, "Cargo"),
    (273332560, "Fishing Vessel"), (273351680, "Fishing Vessel"), (273398460, "Fishing Vessel"),
    (273425330, "Fishing Vessel"), (273432340, "Other Type"), (273448380, "Fishing Vessel"),
    (277558000, "Fishing Vessel"), (982570113, "Other Type"), (244820055, "Cargo"),
    (273354540, "Fishing Vessel"), (273431440, "Fishing Vessel"), (273436210, "Fishing Vessel"),
    (273437550, "Fishing Vessel"), (273439220, "Research Vessel"), (273550600, "Fishing Vessel"),
    (231770000, "Other Type"), (257046460, "Fishing Vessel"), (257071530, "Other Type"),
    (258874000, "Fishing Vessel"), (259616000, "Fishing Vessel"), (273380430, "Fishing Vessel"),
    (273419350, "Cargo"), (273424080, "Fishing Vessel"), (273428090, "Cargo"),
    (244810617, "Cargo"), (258186000, "Fishing Vessel"), (273355780, "Fishing Vessel"),
    (273431310, "Fishing Vessel"), (257043030, "Resolution 18"), (257271600, "Fishing Vessel"),
    (257508500, "Other Type"), (257656000, "Fishing Vessel"), (257806000, "Fishing Vessel"),
    (273293480, "Fishing Vessel"), (273845800, "Fishing Vessel"), (273895100, "Fishing Vessel"),
    (231219000, "Passenger Ship"), (259217000, "Fishing Vessel"), (273218630, "Cargo"),
    (273537800, "Fishing Vessel"), (273891000, "Fishing Vessel")
]


general_categories = {
    "Tanker": "Commercial",
    "Cargo": "Commercial",
    "Supply Ship": "Commercial",
    "Fishing Vessel": "Fishing",
    "Yacht": "Recreational",
    "Passenger Ship": "Passenger",
    "HSC": "Passenger",
    "Pleasure Craft": "Recreational",
    "Sailing Vessel": "Recreational",
    "Research Vessel": "Service",
    "Patrol Vessel": "Service",
    "Military Ops": "Service",
    "Pilot": "Service",
    "Port tender": "Service",
    "Ice breaker": "Service",
    "Lighthouse Vessel": "Special",
    "Pollution Control Vessel": "Service",
    "Offshore Support Vessel": "Service",
    "Resolution 18 ship": "Other",
    "Other Type": "Other"
}

df_ships = pd.DataFrame(full_ship_data, columns=["mmsi", "Specific Type"])
df_ships['General Type'] = df_ships['Specific Type'].map(general_categories)

df_ships
